Error calcs for a single member for VWIND

In [1]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import warnings
from scipy import signal 
import glob as glob

from ecpaper_utils import bootstrap_utils as boot
from ecpaper_utils import filter_utils as filt
from ecpaper_utils import averaging_utils as avg

importlib.reload(boot)
importlib.reload(avg)

warnings.filterwarnings('ignore')

In [2]:
vwind_path="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

In [4]:
#le_models = [x.split("_")[1].split(".")[0] for x in sorted(glob.glob(vwind_path+"/v300*.nc"))] ## parse files to get model names ###doesn't work now we have v300 for the obs
le_models = ['CanESM2','CESM1-CAM5','CSIRO-Mk3-6-0','GFDL-CM3','MPI-ESM']
le_dat = []

for le in le_models:
    print(le)
    ## vwind
    ds = xr.open_dataset(vwind_path+"v300_"+le+".nc")
    ## precip
    dsp = xr.open_dataset(vwind_path+"pr_"+le+".nc").rename({"Member": "member"})
    ## u100
    dsu = xr.open_dataset(vwind_path+"u100_"+le+".nc")
    ds = xr.merge([ds, dsp, dsu]).assign_coords({"Model": le})
    le_dat.append(ds)

CanESM2
CESM1-CAM5
CSIRO-Mk3-6-0
GFDL-CM3
MPI-ESM


In [11]:
print(ds)

<xarray.Dataset>
Dimensions:    (lat: 181, lon: 360, member: 100)
Coordinates:
  * member     (member) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
  * lat        (lat) float64 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon        (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
    Model      <U7 'MPI-ESM'
Data variables:
    vpast_djf  (member, lat, lon) float64 -0.03625 -0.03625 ... 0.006762
    vfut_djf   (member, lat, lon) float64 -0.04518 -0.04518 ... 0.01946 0.01946
    prp_west   (member) float64 4.638 4.773 5.127 4.751 ... 4.782 5.033 4.912
    prf_west   (member) float64 4.75 4.48 4.52 4.811 ... 4.788 4.453 4.715 4.663
    prp_south  (member) float64 2.077 2.084 2.079 1.948 ... 2.045 1.897 2.063
    prf_south  (member) float64 1.745 2.017 1.982 1.946 ... 1.914 1.875 2.003
    prp_mex    (member) float64 1.529 1.391 1.384 1.285 ... 1.508 1.392 1.298
    prf_mex    (member) float64 0.9337 1.102 1.066 0.9528 ... 1.03 0.9757 0.9605
    upast_djf 

In [7]:
#le_dat = xr.concat(le_dat, dim = "Model")

TypeError: can only concatenate xarray Dataset and DataArray objects, got <class 'str'>

Calculate the ensemble mean values for each LE

In [9]:
le_vdiff = []
le_vamp = []
le_pdiff_west = []
le_pdiff_south = []
le_pdiff_mex = []
le_udiff = []

for le in le_models: 
    print(le)
    ds = le_dat.sel(Model = le).dropna(dim = "member", how = "all").mean(dim = "member")
    vpast_eddy = ds["vpast_djf"] - ds["vpast_djf"].mean("lon")
    vfut_eddy = ds["vfut_djf"] - ds["vfut_djf"].mean("lon")
    vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
    vfutsw = avg.cosweightlonlat(vfut_eddy, 250, 265, 25, 37)   
    le_vdiff.append((vfutsw - vpastsw).values)
    
    # stationary wave ampltiude index
    vpast_k4to32 = filt.filterk(ds["vpast_djf"], 4, 32)
    vpast_amp = np.sqrt(avg.cosweightlonlat(vpast_k4to32**2, 200., 300., 20., 40.))
    le_vamp.append(vpast_amp.values)
        
    ## change in precipitation
    le_pdiff_west.append((ds["prf_west"] - ds["prp_west"]).values)
    le_pdiff_south.append((ds["prf_south"] - ds["prp_south"]).values)
    le_pdiff_mex.append((ds["prf_mex"] - ds["prp_mex"]).values)
    
    ## change in u100
    up = avg.cosweightlat(ds["upast_djf"],20,40)
    uf = avg.cosweightlat(ds["ufut_djf"],20, 40)
    le_udiff.append((uf - up).values)

CanESM2
CESM1-CAM5
CSIRO-Mk3-6-0
GFDL-CM3
MPI-ESM


Calculate errors for a single member.

In [10]:
nboots=1000
nmemp=1
nmemf=1

boots_vdiff = []
boots_udiff = []
boots_vamp = []
boots_pdiff_west = []
boots_pdiff_south = []
boots_pdiff_mex = []

for j, le in enumerate(le_models):
        
    ds = le_dat.sel(Model = le).dropna(dim = "member", how = "all")
        
    ## bootstrapped V300
    vpast = boot.bootgen_multimem(ds["vpast_djf"], nmemp, seed=3, nboots = nboots)
    vfut = boot.bootgen_multimem(ds["vfut_djf"], nmemf, seed=3, nboots = nboots)
        
    vpast_eddy = vpast - vpast.mean("lon")
    vfut_eddy = vfut - vfut.mean("lon")
    vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
    vfutsw = avg.cosweightlonlat(vfut_eddy, 250, 265, 25, 37)
        
    ## change in v300 SW
    boots_vdiff.append((vfutsw - vpastsw).values - le_vdiff[j])
        
    # stationary wave ampltiude index
    vpast_k4to32 = filt.filterk(vpast, 4, 32)
    vpast_amp = np.sqrt(avg.cosweightlonlat(vpast_k4to32**2, 200., 300., 20., 40.))
        
    boots_vamp.append(vpast_amp.values - le_vamp[j])
        
    ## u100 difference
    upast = boot.bootgen_multimem(ds["upast_djf"], nmemp, seed = 3, nboots = nboots)
    upast = avg.cosweightlat(upast,20,40)
    ufut = boot.bootgen_multimem(ds["ufut_djf"], nmemf, seed = 3, nboots = nboots)
    ufut = avg.cosweightlat(ufut,20, 40)
    boots_udiff.append(ufut.values - upast.values - le_udiff[j])
        
    ## calculate precip changes for three regions
    prp = boot.bootgen_multimem(ds["prp_west"], nmemp, seed = 3, nboots = nboots)
    prf = boot.bootgen_multimem(ds["prf_west"], nmemf, seed = 3, nboots = nboots)
    boots_pdiff_west.append(prf.values - prp.values - le_pdiff_west[j])
        
    prp = boot.bootgen_multimem(ds["prp_south"], nmemp, seed = 3, nboots = nboots)
    prf = boot.bootgen_multimem(ds["prf_south"], nmemf, seed = 3, nboots = nboots)
    boots_pdiff_south.append(prf.values - prp.values - le_pdiff_south[j])
        
    prp = boot.bootgen_multimem(ds["prp_mex"], nmemp, seed = 3, nboots = nboots)
    prf = boot.bootgen_multimem(ds["prf_mex"], nmemf, seed = 3, nboots = nboots)
    boots_pdiff_mex.append(prf.values - prp.values - le_pdiff_mex[j])
    

boots_vdiff = np.concatenate(boots_vdiff)
boots_udiff = np.concatenate(boots_udiff)
boots_vamp = np.concatenate(boots_vamp)
boots_pdiff_west = np.concatenate(boots_pdiff_west)
boots_pdiff_south = np.concatenate(boots_pdiff_south)
boots_pdiff_mex = np.concatenate(boots_pdiff_mex)
    
sig_vdiff = np.std(boots_vdiff)
sig_udiff = np.std(boots_udiff)
sig_psi = np.std(boots_vamp)
sig_pdiff_west = np.std(boots_pdiff_west)
sig_pdiff_south = np.std(boots_pdiff_south)
sig_pdiff_mex = np.std(boots_pdiff_mex)
    
r_psi_vdiff = np.corrcoef(boots_vamp, boots_vdiff)[0,1]
r_udiff_vdiff = np.corrcoef(boots_udiff, boots_vdiff)[0,1]
r_psi_prwest = np.corrcoef(boots_vamp, boots_pdiff_west)[0,1]
r_psi_prsouth = np.corrcoef(boots_vamp, boots_pdiff_south)[0,1]
r_psi_prmex = np.corrcoef(boots_vamp, boots_pdiff_mex)[0,1]

In [13]:
# 5th to 95th percentile of internal variability range using the first 20 members from each
for j, le in enumerate(le_models):
        
    ds = le_dat.sel(Model = le).dropna(dim = "member", how = "all")
    ds = ds.isel(member=slice(0,20))
    vpast_eddy = ds['vpast_djf'] - ds['vpast_djf'].mean("lon")
    vpastsw = avg.cosweightlonlat(vpast_eddy, 250, 265, 25, 37)
    difs = np.zeros([500])
    difswest = np.zeros([500])
    difssouth = np.zeros([500])
    difsmex = np.zeros([500])
    count=0
    for imem in range(0,20,1):
        indices=np.argwhere(np.arange(0,20,1) !=imem)
        for imem2 in indices:
        #for imem2 in range(imem+1,20,1):
            difs[count] = vpastsw[imem2]-vpastsw[imem]
            difswest[count] = ds['prp_west'][imem2] - ds['prp_west'][imem]
            difssouth[count] = ds['prp_south'][imem2] - ds['prp_south'][imem]
            difsmex[count] = ds['prp_mex'][imem2] - ds['prp_mex'][imem]
            count=count+1

    difs = difs[0:count]
    difswest = difswest[0:count]
    difssouth = difssouth[0:count]
    difsmex = difsmex[0:count]
    if (j == 0):
        difsall = np.zeros([5*count])
        difswestall = np.zeros([5*count])
        difssouthall = np.zeros([5*count])
        difsmexall = np.zeros([5*count])
            
    difsall[j*count:(j+1)*count]=difs[:]
    difswestall[j*count:(j+1)*count]=difswest[:]
    difssouthall[j*count:(j+1)*count]=difssouth[:]
    difsmexall[j*count:(j+1)*count]=difsmex[:]
    
ivmin95=np.percentile(difsall,2.5)
ivmax95=np.percentile(difsall,97.5)
ivmin95west=np.percentile(difswestall,2.5)
ivmax95west=np.percentile(difswestall,97.5)
ivmin95south=np.percentile(difssouthall,2.5)
ivmax95south=np.percentile(difssouthall,97.5)
ivmin95mex=np.percentile(difsmexall,2.5)
ivmax95mex=np.percentile(difsmexall,97.5)

ivmin95 = xr.DataArray(ivmin95, name='ivmin95')
ivmax95 = xr.DataArray(ivmax95, name='ivmax95')
ivmin95west = xr.DataArray(ivmin95west, name='ivmin95')
ivmax95west = xr.DataArray(ivmax95west, name='ivmax95')
ivmin95south = xr.DataArray(ivmin95south, name='ivmin95')
ivmax95south = xr.DataArray(ivmax95south, name='ivmax95')
ivmin95mex = xr.DataArray(ivmin95mex, name='ivmin95')
ivmax95mex = xr.DataArray(ivmax95mex, name='ivmax95')




In [15]:
error_1mem = xr.Dataset({'sig_vdiff': xr.DataArray(sig_vdiff, name='sig_vdiff'), 
                        'sig_psi': xr.DataArray(sig_psi, name='sig_psi'), 
                        'sig_udiff': xr.DataArray(sig_udiff, name='sigg_udiff'),
                        'sig_pdiff_west': xr.DataArray(sig_pdiff_west, name='sigg_pdiff_west'), 
                        'sig_pdiff_south': xr.DataArray(sig_pdiff_south, name='sig_pdiff_south'), 
                        'sig_pdiff_mex': xr.DataArray(sig_pdiff_mex, name='sig_pdiff_mex'), 
                        'r_psi_vdiff': xr.DataArray(r_psi_vdiff, name='r_psi_vdiff'), 
                         'r_udiff_vdiff': xr.DataArray(r_udiff_vdiff, name='r_udiff_vdiff'),
                        'r_psi_prwest': xr.DataArray(r_psi_prwest, name='r_psi_prwest'), 
                        'r_psi_prsouth': xr.DataArray(r_psi_prsouth, name='r_psi_prsouth'), 
                        'r_psi_prmex': xr.DataArray(r_psi_prmex, name='r_psi_prmex'),
                        'ivmin95': xr.DataArray(ivmin95),
                        'ivmax95': xr.DataArray(ivmax95),
                        'ivmin95west': xr.DataArray(ivmin95west),
                        'ivmax95west': xr.DataArray(ivmax95west),
                        'ivmin95south': xr.DataArray(ivmin95south),
                        'ivmax95south': xr.DataArray(ivmax95south),
                        'ivmin95mex': xr.DataArray(ivmin95mex),
                        'ivmax95mex': xr.DataArray(ivmax95mex)})

error_1mem.to_netcdf("./DATA/errors_1mem.nc")